<a href="https://colab.research.google.com/github/mswae/first-mayoral-hackathon/blob/main/peopulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Peopulse**: Public Concern Early Warning System and Decision Support

## Import Libraries

In [ ]:
!pip install -q transformers datasets evaluate

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re #regular expression; for text manipulation
import evaluate #for evaluation metrics

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import hdbscan

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

ds_name = "/content/drive/MyDrive/Datasets/peopulse-dataset-2.csv"
df = pd.read_csv(ds_name)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Model from Hugging Face

In [ ]:
#loads RoBERTa-Tagalog
model_name = "jcblaise/roberta-tagalog-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model_rtb = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=9)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Helper Functions

In [ ]:
# --- preprocessing helper functions ---

def clean_data(text):

  text = text.lower()
  text = re.sub(r"http\S+|www\S+", "", text)  #remove URLs
  text = re.sub(r"@\w+", "", text)  #remove @mentions
  text = re.sub(r"u/\w+", "", text)  #remove u/mentions
  text = re.sub(r"#(\w+)", r"\1", text)  #remove hashtag symbols but keep the word
  text = re.sub(r"[^\w\s!?.,'’“”-]", " ", text)  #keep only basic punctuation
  text = re.sub(r"\s+", " ", text).strip()

  return text

def tokenize_fn(batch, tokenizer, text_col="text", label_col="category"):

  tokens = tokenizer(batch[text_col], truncation=True, padding="max_length", max_length=128    )
  tokens["labels"] = batch[label_col]

  return tokens

# --- evaluation helper function ---

f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]
    precision = precision_metric.compute(predictions=preds, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=preds, references=labels, average="weighted")["recall"]

    results = {
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

    return results

## Stratify Data for Training and Testing (80-20)

In [ ]:
label_col = "category" #replace with the actual name of your label column

le = LabelEncoder()
df[label_col] = le.fit_transform(df[label_col])

X = df.drop(columns=[label_col])
y = df[label_col]

#perform an 80-20 stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Original dataset shape: {y.value_counts(normalize=True)}")
print(f"Train set shape: {y_train.value_counts(normalize=True)}")
print(f"Test set shape: {y_test.value_counts(normalize=True)}")

train_df = pd.DataFrame({
    "text": X_train["text"].apply(clean_data),
    "category": y_train
})

test_df = pd.DataFrame({
    "text": X_test["text"].apply(clean_data),
    "category": y_test
})

#convert to HF dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

#tokenization
train_dataset = train_ds.map(
    lambda batch: tokenize_fn(batch, tokenizer),
    batched=True,
    remove_columns=train_ds.column_names #removes columns after tokenization
)

test_dataset = test_ds.map(
    lambda batch: tokenize_fn(batch, tokenizer),
    batched=True,
    remove_columns=test_ds.column_names #removes columns after tokenization
)

Original dataset shape: category
8    0.111111
2    0.111111
7    0.111111
4    0.111111
5    0.111111
1    0.111111
0    0.111111
3    0.111111
6    0.111111
Name: proportion, dtype: float64
Train set shape: category
8    0.111111
4    0.111111
3    0.111111
5    0.111111
7    0.111111
1    0.111111
2    0.111111
6    0.111111
0    0.111111
Name: proportion, dtype: float64
Test set shape: category
8    0.111111
7    0.111111
6    0.111111
5    0.111111
3    0.111111
1    0.111111
4    0.111111
0    0.111111
2    0.111111
Name: proportion, dtype: float64


Map:   0%|          | 0/2880 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

## **Model Training**

In [ ]:
# args = TrainingArguments(
#     output_dir=f"./results/RoBERTa-Tagalog",
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     eval_strategy="steps",
#     eval_steps=50,
#     save_strategy="steps",
#     save_steps=50,
#     logging_steps=50,
#     logging_dir=f"./logs/RoBERTa-Tagalog",
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     report_to="none"
# )

# trainer = Trainer(
#     model=model_rtb,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],

# )

# trainer.train()

/tmp/ipython-input-1272334242.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,F1,Precision,Recall
50,1.898300,0.972418,0.703109,0.778315,0.719444
100,0.545900,0.108364,0.976247,0.976536,0.976389
150,0.100000,0.015107,0.998611,0.998628,0.998611


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
trainer.state.best_model_checkpoint

'./results/RoBERTa-Tagalog/checkpoint-150'

## Save Model

In [ ]:
# save_path = f"./saved_models/RoBERTa-Tagalog"

# trainer.save_model(save_path)  #saves model weights, config, tokenizer, etc.
# tokenizer.save_pretrained(save_path)

# print(f"Model saved to {save_path}")

Model saved to ./saved_models/RoBERTa-Tagalog


In [ ]:
# # pushing the model to HF
# from huggingface_hub import login, HfApi
# login()

# HF_USERNAME = "sdbrgo"
# # SETTING = "POC" #VAL/TEST/POC
# MODEL = "RoBERTa-Tagalog"

# api = HfApi()

# api.upload_folder(
#     folder_path=f"./saved_models/{MODEL}",
#     repo_id=f"{HF_USERNAME}/roberta-tagalog-public-concern-classifier",
#     repo_type="model"
# )

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/sdbrgo/roberta-tagalog-public-concern-classifier/commit/4b5fa981fe0dadb3df0dc806c22029e5f60bff05', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4b5fa981fe0dadb3df0dc806c22029e5f60bff05', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sdbrgo/roberta-tagalog-public-concern-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='sdbrgo/roberta-tagalog-public-concern-classifier'), pr_revision=None, pr_num=None)

# Peopulse Ingestion Pipeline

### Define Helper Functions

In [2]:
# --- load dataset ---
def load_raw_data(file):
  df = pd.read_csv(path)
  df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
  return df.dropna(subset=["text", "timestamp"])

# --- window selector ---
def filter_current_window(df, lookback_days=30):
  now = pd.Timestamp.now(tz="UTC")
  start = now - pd.Timedelta(days=lookback_days)

  window_df = df[
      (df["timestamp"] >= start) &
      (df["timestamp"] < now)
  ].copy()

  return window_df

# --- cleans "text" column only ---
def clean_text_column(df, text_col="text"):
  df = df.copy()
  df[text_col] = df[text_col].astype(str).apply(clean_data)
  return df

# --- fills out rows with missing category ---
def classify_missing_categories(df, model, tokenizer, text_col="text", cat_col="category"):
  df = df.copy()

  mask = df[cat_col].isna() | (df[cat_col] == "")
  if not mask.any():
      return df  # nothing to do

  texts = df.loc[mask, text_col].tolist()

  inputs = tokenizer(
      texts,
      truncation=True,
      padding=True,
      max_length=128,
      return_tensors="pt"
  )

  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      preds = outputs.logits.argmax(dim=-1).cpu().numpy()

  df.loc[mask, cat_col] = preds
  df.loc[mask, "category_source"] = "predicted"
  df.loc[~mask, "category_source"] = "original"

  return df

## Main

In [ ]:
peopulse_model = "sdbrgo/roberta-tagalog-public-concern-classifier"
tokenizer = AutoTokenizer.from_pretrained(peopulse_model)

df = load_raw_data(PATH)
window_df = filter_current_window(df, lookback_days=30)
window_df = clean_text_column(window_df)
window_df = classify_missing_categories(
    window_df,
    model=peopulse_model,
    tokenizer=tokenizer
)